# 异步IO-传输层和协议层

## 参考文献

* https://docs.python.org/3/library/asyncio-protocol.html

```
At the highest level, the transport is concerned with how bytes are transmitted, while the protocol determines which bytes to transmit (and to some extent when).

A different way of saying the same thing: a transport is an abstraction for a socket (or similar I/O endpoint) while a protocol is an abstraction for an application, from the transport’s point of view.

Yet another view is the transport and protocol interfaces together define an abstract interface for using network I/O and interprocess I/O.

There is always a 1:1 relationship between transport and protocol objects: the protocol calls transport methods to send data, while the transport calls protocol methods to pass it data that has been received.
```


* 从更高的level看，传输层关心的是byte如何传输，协议层哪些byte可以传输；
* 换一种表达方式，传输层是socket的抽象，协议层是应用的抽象；
* 从另外一个角度看，传输层和协议层接口一起定义了网络IO的抽象接口；
* 传输层和协议层的关系，协议层调用传输层接口发送数据，传输层调用协议层接口把收到数据传输到应用。

## 代码示例

### Echo TCP Server

In [1]:
# encoding: utf8

import asyncio
from asyncio import transports
from asyncio import AbstractServer


class EchoServerProtocol(asyncio.Protocol):

    def connection_made(self, transport: transports.BaseTransport) -> None:
        peer_name = transport.get_extra_info('peername')
        print(f'Connection from {peer_name}')
        self.transport = transport  # type: transports.BaseTransport

    def data_received(self, data: bytes) -> None:
        message = data.decode()
        print(f'Data received: {message}')

        print(f'Send {message}')
        self.transport.write(data)

        print('Close the client socket')
        self.transport.close()


async def main():
    loop = asyncio.get_event_loop()

    server = await loop.create_server(lambda: EchoServerProtocol(), 
                                      host='127.0.0.1', port=9999)  # type: AbstractServer

    async with server:
        await server.serve_forever()


# if __name__ == '__main__':
#     asyncio.run(main())


### Echo TCP Client

In [2]:
# encoding: utf8

import asyncio
from asyncio import Transport
from asyncio import Future


class EchoServerProtocol(asyncio.Protocol):

    def __init__(self, message: str, on_conn_lost: Future):
        self.message = message
        self.on_conn_lost = on_conn_lost

    def connection_made(self, transport: Transport) -> None:
        transport.write(self.message.encode())
        print(f'Data sent: {self.message}')

    def data_received(self, data: bytes) -> None:
        print(f'Data received: {data.decode()}')

    def connection_lost(self, exc: Exception) -> None:
        print('The server close the connection')
        self.on_conn_lost.set_result(True)


async def main():
    loop = asyncio.get_event_loop()

    on_conn_lost = loop.create_future()     # type: asyncio.Future
    message = 'Hello World!'

    transport, protocol = await loop.create_connection(lambda: EchoServerProtocol(message, on_conn_lost),
                                                       host='127.0.0.1', port=9999)

    try:
        await on_conn_lost
    finally:
        transport.close()


# if __name__ == '__main__':
#     asyncio.run(main())
